# Part 1: Intro to Apache Spark

1. Start a Spark Session

`pyspark.sql` is a PySpark module that provides tools for working with structured data using DataFrames and SQL-like queries.

In [ ]:
from pyspark.sql import SparkSession

# Start Spark
spark = SparkSession.builder.appName("Biostats Analysis").getOrCreate()

# Load CSV file
df = spark.read.csv("Biostats.csv", header=True, inferSchema=True)

# View first rows
df.show(5)

+----+---+---+----------+-----------+
|Name|Sex|Age|Height(in)|Weight(lbs)|
+----+---+---+----------+-----------+
|Alex|  M| 41|        74|        170|
|Bert|  M| 42|        68|        166|
|Dave|  M| 32|        70|        155|
|Dave|  M| 39|        72|        167|
|Elly|  F| 30|        66|        124|
+----+---+---+----------+-----------+
only showing top 5 rows



2. Explore schema

In [ ]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Height(in): integer (nullable = true)
 |-- Weight(lbs): integer (nullable = true)



3. Select Columns

In [ ]:
df.select("Name", "Age", "Sex").show()

+----+---+---+
|Name|Age|Sex|
+----+---+---+
|Alex| 41|  M|
|Bert| 42|  M|
|Dave| 32|  M|
|Dave| 39|  M|
|Elly| 30|  F|
|Fran| 33|  F|
|Gwen| 26|  F|
|Hank| 30|  M|
|Luke| 53|  M|
|Jake| 32|  M|
|Kate| 47|  F|
|Luke| 34|  M|
|Myra| 23|  F|
|Neil| 36|  M|
|Omar| 38|  M|
|Page| 31|  F|
|Luke| 29|  M|
|Ruth| 28|  F|
+----+---+---+



4. Filter: People older than 40

In [ ]:
df.filter(df["Age"] > 40).show()

+----+---+---+----------+-----------+
|Name|Sex|Age|Height(in)|Weight(lbs)|
+----+---+---+----------+-----------+
|Alex|  M| 41|        74|        170|
|Bert|  M| 42|        68|        166|
|Luke|  M| 53|        72|        175|
|Kate|  F| 47|        69|        139|
+----+---+---+----------+-----------+



5. Sort by Weight descending order.

In [ ]:
df.orderBy(df["Weight(lbs)"].desc()).show()

+----+---+---+----------+-----------+
|Name|Sex|Age|Height(in)|Weight(lbs)|
+----+---+---+----------+-----------+
|Luke|  M| 29|        71|        176|
|Luke|  M| 53|        72|        175|
|Alex|  M| 41|        74|        170|
|Dave|  M| 39|        72|        167|
|Bert|  M| 42|        68|        166|
|Luke|  M| 34|        72|        163|
|Neil|  M| 36|        75|        160|
|Hank|  M| 30|        71|        158|
|Dave|  M| 32|        70|        155|
|Omar|  M| 38|        70|        145|
|Jake|  M| 32|        69|        143|
|Kate|  F| 47|        69|        139|
|Page|  F| 31|        67|        135|
|Ruth|  F| 28|        65|        131|
|Elly|  F| 30|        66|        124|
|Gwen|  F| 26|        64|        121|
|Fran|  F| 33|        66|        115|
|Myra|  F| 23|        62|         98|
+----+---+---+----------+-----------+



6. Group by Sex and Count

In [ ]:
df.groupBy("Sex").count().show()

+---+-----+
|Sex|count|
+---+-----+
|  F|    7|
|  M|   11|
+---+-----+



7. Average Weight by Sex


In [ ]:
df.groupBy("Sex").avg("Weight(lbs)").show()


+---+------------------+
|Sex|  avg(Weight(lbs))|
+---+------------------+
|  F|123.28571428571429|
|  M|161.63636363636363|
+---+------------------+



8. Before running SQL queries, register the DataFrame as a temporary view. Now you can query it like a regular SQL table.


In [ ]:
df.createOrReplaceTempView("biostats")

9.  Select Name, Age, Sex

In [ ]:
spark.sql("SELECT Name, Age, Sex FROM biostats").show()

+----+---+---+
|Name|Age|Sex|
+----+---+---+
|Alex| 41|  M|
|Bert| 42|  M|
|Dave| 32|  M|
|Dave| 39|  M|
|Elly| 30|  F|
|Fran| 33|  F|
|Gwen| 26|  F|
|Hank| 30|  M|
|Luke| 53|  M|
|Jake| 32|  M|
|Kate| 47|  F|
|Luke| 34|  M|
|Myra| 23|  F|
|Neil| 36|  M|
|Omar| 38|  M|
|Page| 31|  F|
|Luke| 29|  M|
|Ruth| 28|  F|
+----+---+---+



10. Filter: People older than 40

In [ ]:
spark.sql("SELECT * FROM biostats WHERE Age > 40").show()

+----+---+---+----------+-----------+
|Name|Sex|Age|Height(in)|Weight(lbs)|
+----+---+---+----------+-----------+
|Alex|  M| 41|        74|        170|
|Bert|  M| 42|        68|        166|
|Luke|  M| 53|        72|        175|
|Kate|  F| 47|        69|        139|
+----+---+---+----------+-----------+



# Part 2: Data cleaning with Spark

1. Load the CSV

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DataFrameMapReduce").getOrCreate()
df = spark.read.csv("Movies.csv", header=True, inferSchema=True)

2. Clean the data

In [ ]:
from pyspark.sql.functions import lower, trim

df = df.withColumn("Genre_clean", trim(lower(df["Genre"])))

from pyspark.sql.functions import when

df = df.withColumn(
    "Genre_fixed",
    when(df["Genre_clean"] == "comdy", "comedy")
    .when(df["Genre_clean"] == "romence", "romance")
    .when(df["Genre_clean"] == "romance", "romance")
    .when(df["Genre_clean"] == "comedy", "comedy")
    .when(df["Genre_clean"] == "action", "action")
    .when(df["Genre_clean"] == "drama", "drama")
    .when(df["Genre_clean"] == "animation", "animation")
    .when(df["Genre_clean"] == "fantasy", "fantasy")
    .otherwise(df["Genre_clean"])
)

3. Run the map reduce process

In [ ]:
df.groupBy("Genre_clean").count().show()

+-----------+-----+
|Genre_clean|count|
+-----------+-----+
|    fantasy|    1|
|     action|    1|
|  animation|    4|
|      comdy|    1|
|    romence|    1|
|      drama|   13|
|    romance|   13|
|     comedy|   42|
+-----------+-----+



4. Group in ascending order

In [ ]:
df.groupBy("Genre_fixed").count().orderBy("count", ascending=False).show()

+-----------+-----+
|Genre_fixed|count|
+-----------+-----+
|     comedy|   43|
|    romance|   14|
|      drama|   13|
|  animation|    4|
|    fantasy|    1|
|     action|    1|
+-----------+-----+



5. Save as CSV

In [ ]:
genre_counts = df.groupBy("Genre_fixed").count().orderBy("count", ascending=False)
genre_counts.write.csv("output", header=True, mode="overwrite")

6. Load the movies file

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MoviesCleaning").getOrCreate()
df = spark.read.csv("Movies.csv", header=True, inferSchema=True)
df.printSchema()


root
 |-- Film: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Lead Studio: string (nullable = true)
 |-- Audience score %: integer (nullable = true)
 |-- Profitability: double (nullable = true)
 |-- Rotten Tomatoes %: integer (nullable = true)
 |-- Worldwide Gross: string (nullable = true)
 |-- Year: integer (nullable = true)



7. Drop rows with missing key fields

In [ ]:
df = df.dropna(subset=["Film", "Genre", "Audience score %", "Profitability"])

8. Clean and standardize names

In [ ]:
from pyspark.sql.functions import trim, lower, when, col

# Create cleaned genre
df = df.withColumn("Genre_clean", trim(lower(col("Genre"))))

# Fix typos
df = df.withColumn("Genre_fixed",
    when(col("Genre_clean") == "comdy", "comedy")
    .when(col("Genre_clean") == "romence", "romance")
    .otherwise(col("Genre_clean"))
)

9. Clean and convert Worldwide Gross to numeric

In [ ]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("Worldwide_Gross_Clean",
    regexp_replace("Worldwide Gross", "[$,]", "").cast("float")
)

10. Convert to data types

In [ ]:
df = df.withColumn("Profitability", col("Profitability").cast("float"))
df = df.withColumn("Audience_Score", col("Audience score %").cast("float"))

11. Drop duplicated

In [ ]:
df = df.dropDuplicates(["Film", "Year"])

12. Check final schema

In [ ]:
df.printSchema()
df.select("Film", "Genre_fixed", "Audience_Score", "Profitability", "Worldwide_Gross_Clean").show(5)


root
 |-- Film: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Lead Studio: string (nullable = true)
 |-- Audience score %: integer (nullable = true)
 |-- Profitability: float (nullable = true)
 |-- Rotten Tomatoes %: integer (nullable = true)
 |-- Worldwide Gross: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Genre_clean: string (nullable = true)
 |-- Genre_fixed: string (nullable = true)
 |-- Worldwide_Gross_Clean: float (nullable = true)
 |-- Audience_Score: float (nullable = true)

+--------------------+-----------+--------------+-------------+---------------------+
|                Film|Genre_fixed|Audience_Score|Profitability|Worldwide_Gross_Clean|
+--------------------+-----------+--------------+-------------+---------------------+
|(500) Days of Summer|     comedy|          81.0|        8.096|                60.72|
|          27 Dresses|     comedy|          71.0|    5.3436217|               160.31|
|  A Dangerous Method|      drama|    

13. Save to a new directory with a single CSV file


In [ ]:
df.coalesce(1).write.csv("cleaned_movies_output", header=True, mode="overwrite")

# Part 3: Exercises

1. Show the first 5 records

* Question: Display the first five rows of the dataset.

💡 Hint: Use .show(5) after reading the CSV with spark.read.csv().

2. Display the schema of the dataset

* Question: What are the data types of each column?

💡 Hint: Use .printSchema() to inspect structure and types.

3. Count how many startups are from each country

* Question: Find the number of unicorns per country.

💡 Hint: Use groupBy("Country").count() and sort with .orderBy(...).

4. What are the top 10 most valuable startups?

* Question: List the 10 companies with the highest valuation.

💡 Hint: Make sure Valuation is numeric → use .cast("float"), then .orderBy(...).

5. Filter all startups in the Fintech industry

* Question: Find all rows where the industry is “Fintech”.

💡 Hint: Use .filter(col("Industry") == "Fintech").

6. Count how many unicorns each city has

* Question: Group by City and count the number of entries.

💡 Hint: groupBy("City").count() — sort to see top cities.

7. Count how many unicorns were founded each year

* Question: Count unicorns per year from the Date column.

💡 Hint:

Use to_date("Date", "M/d/yyyy")

Extract year() to a new column

Group and count by Year

8. What’s the average valuation per industry?

* Question: Calculate and rank average startup valuation by industry.

💡 Hint:

Cast Valuation to float

Use groupBy("Industry").avg(...)

Order by the result

9. Create a new column to flag U.S. startups

* Question: Add a column Is_USA to mark startups from the U.S.

💡 Hint: Use withColumn("Is_USA", col("Country") == "United States")

10. Save the cleaned DataFrame to a CSV file
* Question: Write the cleaned and transformed DataFrame to disk.

💡 Hint: Use coalesce(1).write.csv(..., header=True, mode="overwrite")

# Part 3: Exercises solutions

Download the following data: https://www.kaggle.com/datasets/uzairrehman/world-wide-unicorn-startups

1. Load the dataset

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Unicorns").getOrCreate()

df = spark.read.csv("World_Wide_Unicorn_Startups.csv", header=True, inferSchema=True)

2. Show the first 5 records


In [ ]:
df.show(5)

+--------------------+--------------------+----------+-------+-------+--------------------+--------------------+----+-----+----+
|             Company|           Valuation|      Date|Country|   City|            Industry|           Investors|year|month| day|
+--------------------+--------------------+----------+-------+-------+--------------------+--------------------+----+-----+----+
|           Bytedance|               140.0|  4/7/2017|  China|Beijing|Artificial intell...|0      Sequoia Ca...|NULL| NULL|NULL|
|1      Founders Fund| Draper Fisher Ju...| Rothen...|   NULL|   NULL|                NULL|                NULL|NULL| NULL|NULL|
|2            Khos...|    LowercaseCapital|  capitalG|   NULL|   NULL|                NULL|                NULL|NULL| NULL|NULL|
|3      Institutio...|   Sequoia Capita...|      NULL|   NULL|   NULL|                NULL|                NULL|NULL| NULL|NULL|
|4      Sequoia Ca...|  Blackbird Ventures|    Mat...|   NULL|   NULL|                NULL|      

3. Show the schema of the DataFrame

In [ ]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Valuation: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Investors: string (nullable = true)
 |-- year: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)



4. Count how many startups are from each country

In [ ]:
df.groupBy("Country").count().orderBy("count", ascending=False).show()

+------------------+------+
|           Country| count|
+------------------+------+
|              NULL|870480|
|       Alibaba ...|   936|
|             Go...|   936|
| McKesson Ventures|   936|
|               ...|   936|
|              2021|   480|
|     United States|   477|
|             China|   169|
|              2020|   110|
|              2019|   110|
|              2018|   107|
|             India|    51|
|              2017|    46|
|              2015|    37|
|    United Kingdom|    37|
|           Germany|    23|
|              2016|    22|
|            Israel|    21|
|            France|    19|
|            Brazil|    15|
+------------------+------+
only showing top 20 rows



5. Find the top 10 most valuable startups

In [ ]:
df.orderBy(df["Valuation"].desc()).select("Company", "Valuation").show(10)

+--------------------+--------------------+
|             Company|           Valuation|
+--------------------+--------------------+
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
|857    Hyde Park ...|\tFundersClub. Ba...|
+--------------------+--------------------+
only showing top 10 rows



6. Filter all startups in the Fintech industry

In [ ]:
df.filter(df["Industry"] == "Fintech").show()

+--------------------+---------+----------+--------------+-------------+--------+--------------------+----+-----+----+
|             Company|Valuation|      Date|       Country|         City|Industry|           Investors|year|month| day|
+--------------------+---------+----------+--------------+-------------+--------+--------------------+----+-----+----+
|              Stripe|     95.0| 1/23/2014| United States|San Francisco| Fintech|0      Sequoia Ca...|NULL| NULL|NULL|
|              Klarna|     45.6|12/12/2011|        Sweden|    Stockholm| Fintech|0      Sequoia Ca...|NULL| NULL|NULL|
|             Revolut|     33.0| 4/26/2018|United Kingdom|       London| Fintech|0      Sequoia Ca...|NULL| NULL|NULL|
|              Nubank|     30.0|  3/1/2018|        Brazil|    Sao Paulo| Fintech|0      Sequoia Ca...|NULL| NULL|NULL|
|               Chime|     25.0|  3/5/2019| United States|San Francisco| Fintech|0      Sequoia Ca...|NULL| NULL|NULL|
|                 FTX|     25.0| 7/20/2021|     

7. Count how many unicorns each city has

In [ ]:
import time

s = time.time()

df.groupBy("City").count().orderBy("count", ascending=False)

print(time.time()-s)

s = time.time()

df.groupBy("City").count().orderBy("count", ascending=False).show(10)

print(time.time()-s)


0.020714282989501953
+-------------+------+
|         City| count|
+-------------+------+
|         NULL|874239|
|San Francisco|   134|
|     New York|    81|
|      Beijing|    62|
|           16|    46|
|     Shanghai|    45|
|            1|    43|
|           22|    40|
|           21|    40|
|           13|    39|
+-------------+------+
only showing top 10 rows

2.237185001373291


8. Count unicorns per year

In [ ]:
from pyspark.sql.functions import to_date

df = df.withColumn("ParsedDate", to_date("Date", "M/d/yyyy"))

from pyspark.sql.functions import year

df = df.withColumn("Year", year("ParsedDate"))

df.groupBy("Year").count().orderBy("Year").show()

+----+------+
|Year| count|
+----+------+
|NULL|875160|
|2007|     1|
|2010|     1|
|2011|     2|
|2012|     4|
|2013|     3|
|2014|    13|
|2015|    37|
|2016|    22|
|2017|    46|
|2018|   107|
|2019|   110|
|2020|   110|
|2021|   480|
+----+------+



9. Average valuation per industry

In [ ]:
df = df.withColumn("Valuation_numeric", col("Valuation").cast("float"))

df.groupBy("Industry") \
  .avg("Valuation_numeric") \
  .orderBy("avg(Valuation_numeric)", ascending=False) \
  .show()

+--------------------+----------------------+
|            Industry|avg(Valuation_numeric)|
+--------------------+----------------------+
|            Finttech|                  10.0|
|               Other|     4.812941237991931|
|Artificial intell...|     4.465797102969626|
|             Fintech|    3.8638421033558092|
|Data management &...|     3.380555556880103|
|              Edtech|     3.161851860858776|
|            Hardware|     3.080937512218952|
|Auto & transporta...|     3.079310339072655|
|Internet software...|     2.896766457728997|
|Supply chain, log...|     2.868431374138477|
|E-commerce & dire...|    2.8539215711986317|
|   Consumer & retail|      2.78826089527296|
|              Travel|    2.7384615678053637|
|              Health|    2.6390476207884532|
|       Cybersecurity|     2.622926857413315|
|Mobile & telecomm...|    2.1375675555822014|
|Artificial Intell...|     1.175000011920929|
|                   7|                  NULL|
|                  11|            

10.  Create a new column Is_USA to check if company is from United States

In [ ]:
from pyspark.sql.functions import col

df = df.withColumn("Is_USA", col("Country") == "United States")
df.select("Company", "Country", "Is_USA").show(5)


+--------------------+-------+------+
|             Company|Country|Is_USA|
+--------------------+-------+------+
|           Bytedance|  China| false|
|1      Founders Fund|   NULL|  NULL|
|2            Khos...|   NULL|  NULL|
|3      Institutio...|   NULL|  NULL|
|4      Sequoia Ca...|   NULL|  NULL|
+--------------------+-------+------+
only showing top 5 rows



11. Save the cleaned and enriched dataset to CSV

In [ ]:
df.coalesce(1).write.csv("unicorns_cleaned.csv", header=True, mode="overwrite")

# Part 3: Solutions in Spark SQL

1. Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, year, col

spark = SparkSession.builder.appName("Unicorns").getOrCreate()

# Load and prepare
df = spark.read.csv("World_Wide_Unicorn_Startups.csv", header=True, inferSchema=True)

# Parse date and cast valuation
df = df.withColumn("ParsedDate", to_date("Date", "M/d/yyyy"))
df = df.withColumn("Year", year("ParsedDate"))
df = df.withColumn("Valuation_numeric", col("Valuation").cast("float"))

# Register SQL view
df.createOrReplaceTempView("unicorns")


2. Show the first 5 records

In [ ]:
spark.sql("SELECT * FROM unicorns LIMIT 5").show()

+--------------------+--------------------+----------+-------+-------+--------------------+--------------------+----+-----+----+----------+-----------------+
|             Company|           Valuation|      Date|Country|   City|            Industry|           Investors|Year|month| day|ParsedDate|Valuation_numeric|
+--------------------+--------------------+----------+-------+-------+--------------------+--------------------+----+-----+----+----------+-----------------+
|           Bytedance|               140.0|  4/7/2017|  China|Beijing|Artificial intell...|0      Sequoia Ca...|2017| NULL|NULL|2017-04-07|            140.0|
|1      Founders Fund| Draper Fisher Ju...| Rothen...|   NULL|   NULL|                NULL|                NULL|NULL| NULL|NULL|      NULL|             NULL|
|2            Khos...|    LowercaseCapital|  capitalG|   NULL|   NULL|                NULL|                NULL|NULL| NULL|NULL|      NULL|             NULL|
|3      Institutio...|   Sequoia Capita...|      NUL

3. Show the schema

In [ ]:
df.printSchema()


root
 |-- Company: string (nullable = true)
 |-- Valuation: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Investors: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- ParsedDate: date (nullable = true)
 |-- Valuation_numeric: float (nullable = true)



4. Count startups per country

In [ ]:
spark.sql("""
SELECT Country, COUNT(*) AS num_startups
FROM unicorns
GROUP BY Country
ORDER BY num_startups DESC
""").show()


+------------------+------------+
|           Country|num_startups|
+------------------+------------+
|              NULL|      870480|
|       Alibaba ...|         936|
|             Go...|         936|
| McKesson Ventures|         936|
|               ...|         936|
|              2021|         480|
|     United States|         477|
|             China|         169|
|              2020|         110|
|              2019|         110|
|              2018|         107|
|             India|          51|
|              2017|          46|
|              2015|          37|
|    United Kingdom|          37|
|           Germany|          23|
|              2016|          22|
|            Israel|          21|
|            France|          19|
|            Brazil|          15|
+------------------+------------+
only showing top 20 rows



5. Top 10 most valuable startups

In [ ]:
spark.sql("""
SELECT Company, Valuation_numeric
FROM unicorns
ORDER BY Valuation_numeric DESC
LIMIT 10
""").show()


+----------+-----------------+
|   Company|Valuation_numeric|
+----------+-----------------+
| Bytedance|            140.0|
|    SpaceX|            100.3|
|    Stripe|             95.0|
|    Klarna|             45.6|
|     Canva|             40.0|
| Instacart|             39.0|
|Databricks|             38.0|
|   Revolut|             33.0|
|    Nubank|             30.0|
|Epic Games|             28.7|
+----------+-----------------+



6. Filter startups in Fintech industry

In [ ]:
spark.sql("""
SELECT *
FROM unicorns
WHERE Industry = 'Fintech'
""").show()


+--------------------+---------+----------+--------------+-------------+--------+--------------------+----+-----+----+----------+-----------------+
|             Company|Valuation|      Date|       Country|         City|Industry|           Investors|Year|month| day|ParsedDate|Valuation_numeric|
+--------------------+---------+----------+--------------+-------------+--------+--------------------+----+-----+----+----------+-----------------+
|              Stripe|     95.0| 1/23/2014| United States|San Francisco| Fintech|0      Sequoia Ca...|2014| NULL|NULL|2014-01-23|             95.0|
|              Klarna|     45.6|12/12/2011|        Sweden|    Stockholm| Fintech|0      Sequoia Ca...|2011| NULL|NULL|2011-12-12|             45.6|
|             Revolut|     33.0| 4/26/2018|United Kingdom|       London| Fintech|0      Sequoia Ca...|2018| NULL|NULL|2018-04-26|             33.0|
|              Nubank|     30.0|  3/1/2018|        Brazil|    Sao Paulo| Fintech|0      Sequoia Ca...|2018| NULL

7. Count startups per city

In [ ]:
spark.sql("""
SELECT City, COUNT(*) AS count
FROM unicorns
GROUP BY City
ORDER BY count DESC
LIMIT 10
""").show()


+-------------+------+
|         City| count|
+-------------+------+
|         NULL|874239|
|San Francisco|   134|
|     New York|    81|
|      Beijing|    62|
|           16|    46|
|     Shanghai|    45|
|            1|    43|
|           22|    40|
|           21|    40|
|           13|    39|
+-------------+------+



8. Count unicorns per year


In [ ]:
spark.sql("""
SELECT Year, COUNT(*) AS count
FROM unicorns
GROUP BY Year
ORDER BY Year
""").show()


+----+------+
|Year| count|
+----+------+
|NULL|875160|
|2007|     1|
|2010|     1|
|2011|     2|
|2012|     4|
|2013|     3|
|2014|    13|
|2015|    37|
|2016|    22|
|2017|    46|
|2018|   107|
|2019|   110|
|2020|   110|
|2021|   480|
+----+------+



9. Average valuation per industry

In [ ]:
spark.sql("""
SELECT Industry, AVG(Valuation_numeric) AS avg_valuation
FROM unicorns
GROUP BY Industry
ORDER BY avg_valuation DESC
""").show()

+--------------------+------------------+
|            Industry|     avg_valuation|
+--------------------+------------------+
|            Finttech|              10.0|
|               Other| 4.812941237991931|
|Artificial intell...| 4.465797102969626|
|             Fintech|3.8638421033558092|
|Data management &...| 3.380555556880103|
|              Edtech| 3.161851860858776|
|            Hardware| 3.080937512218952|
|Auto & transporta...| 3.079310339072655|
|Internet software...| 2.896766457728997|
|Supply chain, log...| 2.868431374138477|
|E-commerce & dire...|2.8539215711986317|
|   Consumer & retail|  2.78826089527296|
|              Travel|2.7384615678053637|
|              Health|2.6390476207884532|
|       Cybersecurity| 2.622926857413315|
|Mobile & telecomm...|2.1375675555822014|
|Artificial Intell...| 1.175000011920929|
|                   7|              NULL|
|                  11|              NULL|
|                   3|              NULL|
+--------------------+------------

10. Create Is_USA flag

In [ ]:
spark.sql("""
SELECT Company, Country,
       CASE WHEN Country = 'United States' THEN TRUE ELSE FALSE END AS Is_USA
FROM unicorns
""").show(5)

+--------------------+-------+------+
|             Company|Country|Is_USA|
+--------------------+-------+------+
|           Bytedance|  China| false|
|1      Founders Fund|   NULL| false|
|2            Khos...|   NULL| false|
|3      Institutio...|   NULL| false|
|4      Sequoia Ca...|   NULL| false|
+--------------------+-------+------+
only showing top 5 rows

